# NLP ECI 2019 

## Trabajo Práctico

### Introducción

El objetivo del trabajo práctico es reproducir el resultado obtenido en [Gururangan et al., 2018](https://www.aclweb.org/anthology/N18-2017) en el área del _Natural Language Inference_ obtenido sobre el dataset [SNLI](https://nlp.stanford.edu/projects/snli/). 
En esta tarea se debe responder, dadas dos frases A y B, si B es implicación de A ("entailment"), B es contradictorio con A ("contradiction") o si lo que enuncia B es neutral respecto de A ("neutral"). Se dice que A es la premisa y B es la hipótesis. En este trabajo práctico intentaremos predecir a qué clase pertenece cada una de las hipótesis sin observar la premisa.

### Desarrollo

Para replicar los resultados obtenidos en [Gururangan et al., 2018](https://www.aclweb.org/anthology/N18-2017) haremos uso de un clasificador lineal generalizado implementado en la librería [FastText](https://fasttext.cc/). El mismo comienza por construir los _embeddings_ correspondientes a las palabras del corpus suministrado mediante un algoritmo similar a CBOW, para posteriormente alimentar un clasificador lineal en su entrenamiento con el objetivo de predecir las etiquetas correctas (`entailment`, `neutral` o `contradiction` en nuestro caso). Una descripción completa de la técnica utilizada puede hallarse en [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759), mientras que las referencias de la API de este clasificador pueden hallarse en [train_supervised fasttext python API](https://fasttext.cc/docs/en/python-module.html#train_supervised-parameters)

Comenzaremos por implementar nuestro clasificador _custom_, usando a fasttext como backend, para facilitar las posteriores tareas de seleccion de parámetros.

In [10]:
import os
import csv
import logging

import fasttext
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

logging.basicConfig(level=logging.INFO)

class FastTextClassifier(BaseEstimator, ClassifierMixin):
    """Base classifier for fasttext."""

    def __init__(
        self,
        storage_path='data',
        wordNgrams=2,
        ws=5,
        lr=0.1,
        dim=90,
        epoch=100,
        minCount=2,
        verbose=1):
        """Set instance parameters."""
        self.storage_path = storage_path
        self.wordNgrams = wordNgrams
        self.ws = ws
        self.lr = lr
        self.dim = dim
        self.epoch = epoch
        self.minCount = minCount
        self.verbose = verbose

    def get_dev_and_train(
            self,
            train_data='snli_1.0_train_filtered.jsonl',
            train_labels='snli_1.0_train_gold_labels.csv',
            dev_data='snli_1.0_dev_filtered.jsonl',
            dev_labels='snli_1.0_dev_gold_labels.csv',
            key='pairID'):
        """Preprocess train and dev data."""
        train_data_path = os.path.join(
            self.storage_path, train_data)
        train_labels_path = os.path.join(
            self.storage_path, train_labels)
        dev_data_path = os.path.join(
            self.storage_path, dev_data)
        dev_labels_path = os.path.join(
            self.storage_path, dev_labels)

        train_info = pd.read_json(train_data_path, lines=True)
        dev_info = pd.read_json(dev_data_path, lines=True)

        train_labels = pd.read_csv(train_labels_path)
        dev_labels = pd.read_csv(dev_labels_path)

        train = train_info.set_index(key).join(train_labels.set_index(key))
        dev = dev_info.set_index(key).join(dev_labels.set_index(key))
        train = train[['sentence2', 'gold_label']]
        dev = dev[['sentence2', 'gold_label']]

        df = train.append(dev)

        return df, train, dev

    def preprocess(self, X, lower=True, replace_dict={}):
        X_new = X.copy()
        if lower:
            X_new = X_new.str.lower()
        for k, v in replace_dict.items():
            X_new = X_new.str.replace(k, v)
        return X_new


    def fit(self, X, y):
        buffer_filename = os.path.join(self.storage_path, 'buffer.csv')
        series = X + ' __label__' + y
        series.to_csv(buffer_filename, encoding='utf-8', index=False, header=False)
        logging.info('Fitting classifier...')
        model = fasttext.train_supervised(
            buffer_filename,
            wordNgrams=self.wordNgrams,
            ws=self.ws,
            lr=self.lr,
            dim=self.dim,
            epoch=self.epoch,
            minCount=self.minCount,
            verbose=self.verbose)

        self.model_ = model

        return self

    def predict(self, X):
        y = []
        try:
            getattr(self, "model_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        for sentence in X:
            y.append(
                self.model_.predict(sentence)[0][0].replace('__label__', ''))
        return y

    def score(self, X, y, metric='accuracy', sample_weight=None):
        y_hat = self.predict(X)
        if metric == 'accuracy':
            return accuracy_score(y, y_hat, sample_weight=sample_weight)

El clasificador se inicia con algunos parámetros fijos y se provee un diccionario para pre-procesar el corpus. Los pasos de este pre-procesamiento serán

- Convertir todas las palabras a minúscula
- Eliminar todos los caracteres no alfanuméricos
- Remplazar a los números escritos por sus dígitos

El dataset utilizado para el entrenamiento será el provisto para `train` y será posteriormente validado con `dev`

In [11]:
ftc = FastTextClassifier()

replace_dict = {
    '[^a-z1-9 ]+': '',
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9'}

total, train, dev = ftc.get_dev_and_train()
train.sentence2 = ftc.preprocess(train.sentence2, replace_dict=replace_dict)
dev.sentence2 = ftc.preprocess(dev.sentence2, replace_dict=replace_dict)
total.sentence2 = ftc.preprocess(total.sentence2, replace_dict=replace_dict)

Realizamos un _GridSearch_ para encontrar el mejor conjunto de parámetros para nuestro clasificador mediante _CrossValidation_

In [6]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(FastTextClassifier(epoch=80) , param_grid=dict(
    wordNgrams=[2, 3],
    lr=[0.1, 0.25]))
clf.fit(X=train.sentence2, y=train.gold_label)

INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...
INFO:root:Fitting classifier...


GridSearchCV(cv=None, error_score='raise',
       estimator=FastTextClassifier(dim=90, epoch=80, lr=0.1, minCount=2, storage_path='data',
          verbose=1, wordNgrams=2, ws=5),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'wordNgrams': [2, 3], 'lr': [0.1, 0.25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Y una vez obtenidos los parámetros los guardamos y analizamos el _score_ (_Accuracy_) alcanzado para el mejor de los modelos en el dataset de `dev`

In [12]:
best_ftc = clf.best_estimator_
print('Best model params', best_ftc.get_params())
print('Best model score:', clf.best_score_,
      'Best model score on dev:', best_ftc.score(X=dev.sentence2, y=dev.gold_label))

Best model params {'dim': 90, 'epoch': 80, 'lr': 0.25, 'minCount': 2, 'storage_path': 'data', 'verbose': 1, 'wordNgrams': 2, 'ws': 5}
Best model score: 0.6353767153833412 Best model score on dev: 0.6546433651696809


Con el mismo entrenamos nuestro clasificador ahora sobre el corpus completo

In [13]:
best_ftc.fit(X=total.sentence2, y=total.gold_label)

INFO:root:Fitting classifier...


FastTextClassifier(dim=90, epoch=80, lr=0.25, minCount=2, storage_path='data',
          verbose=1, wordNgrams=2, ws=5)

Y realizamos la predicción final sobre el dataset de testeo. Guardamos el mismo para subirlo a la página de la competencia.

In [15]:
test_data = pd.read_json('data/snli_1.0_test_filtered.jsonl', lines=True)
test_data.sentence2 = ftc.preprocess(test_data.sentence2, replace_dict=replace_dict)
test_data['gold_label'] = best_ftc.predict(test_data.sentence2)
results = test_data[['pairID', 'gold_label']]
results.to_csv('best_estimator_results.csv', index=False)